#Planning Subject to Well-Known Temporal Availability Constraints (MILP Algorithm) : RoboMAX Context.


In [ ]:
%pip install pulp

In [ ]:
import random
from pulp import *

# Linear constraints to ensure that the robots do not overlap
def check_robot_constraints(assignment):
    for i in range(len(assignment)):
        for j in range(i + 1, len(assignment)):
            if assignment[i].robot == assignment[j].robot:
                if assignment[i].tiempo_fin > assignment[j].tiempo_inicio and assignment[j].tiempo_fin > assignment[i].tiempo_inicio:
                    return False
    return True

# Function to obtain the exact time it takes to attend to a patient
def get_exact_attending_time(paciente, begin_time, end_time):
    tiempo_inicio = begin_time
    tiempo_fin = end_time

    ##Assuming the time it takes to deliver the meal is 0

    if paciente.has_food == False:
        tiempo_fin_atencion = tiempo_inicio + paciente.get_attending_time

        if tiempo_fin_atencion > tiempo_fin:
            tiempo_fin_atencion = tiempo_fin
    else:
        tiempo_fin_atencion = tiempo_inicio

    return tiempo_fin_atencion

# Function to solve the problem using linear programming
def solve_optimization_problem(vehiculos, cargadores):

    # Create the problem
    prob = LpProblem("Carga_de_vehiculos", LpMinimize)

    # Decission variables
    assignment_vars = LpVariable.dicts("Assignment", [(paciente.id, robot.id) for paciente in pacientes for robot in robots], 0, 1, LpBinary)

    # Objective function
    prob += lpSum([assignment_vars[(paciente.id, robot.id)] * paciente.get_attending_time() for paciente in pacientes for robot in robots])

    # Constraints
    for paciente in pacientes:
        prob += lpSum([assignment_vars[(paciente.id, robot.id)] for robot in robots]) == 1


    for robot in robots:
        prob += lpSum([assignment_vars[(paciente.id, robot.id)] for paciente in pacientes]) <= 1

    # Solve the problem
    prob.solve()

    # Extract the solution
    best_assignment = []
    for paciente in pacientes:
        for cargador in cargadores:
            if assignment_vars[(paciente.id, robot.id)].value() == 1:
                tiempo_inicio = paciente.available_time_start * assignment_vars[(paciente.id, robot.id)].value()
                tiempo_fin = paciente.available_time_end * assignment_vars[(robot.id, cargador.id)].value()
                tiempo_fin_atencion = get_exact_attending_time(paciente, tiempo_inicio, tiempo_fin)
                best_assignment.append(item_planlist(paciente, tiempo_inicio, tiempo_fin_atencion, robot))


    return best_assignment


file_name = 'pacientes2.txt'
pacientes = create_patients_from_file(file_name)



# Robots inicialization
robots = [Robot(1), Robot(2), Robot(3)]

import time
inicial = time.time()

# Solve the problem
best_assignment= solve_optimization_problem(pacientes, robots)

final = time.time()

# Execution time
tiempo_transcurrido = final - inicial

# Imprimir la mejor solución encontrada

print("Planificación:")
for item in best_assignment:
    print("Vehículo " + str(item.vehiculo.id))
    print("Inicio: " + str(item.tiempo_inicio))
    print("Fin: " + str(item.tiempo_fin))
    print("Cargador " + str(item.cargador.id))
    print("")

print(tiempo_transcurrido)
